In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    # rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, dones, rates

In [4]:
def actor(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('actor', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def generator(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('generator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=state_size)        
        return logits

In [6]:
def discriminator(states, actions, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('discriminator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=1)        
        return logits

In [7]:
def model_loss(state_size, action_size, hidden_size,
               states, actions, next_states, dones, rates):
    actions_logits = actor(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ###############################################
    next_states_logits = generator(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                   action_size=action_size, state_size=state_size)
    next_states_labels = tf.nn.sigmoid(next_states)
    aloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=next_states_logits, 
                                                                    labels=next_states_labels))
    ####################################################
    dQs = discriminator(actions=actions_labels, hidden_size=hidden_size, states=states, action_size=action_size)
    rates = tf.reshape(rates, shape=[-1, 1])
    dloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=dQs, # GAN
                                                                   labels=rates)) # 0-1
    ####################################################
    gQs = discriminator(actions=actions_logits, hidden_size=hidden_size, states=states, action_size=action_size, 
                        reuse=True)
    dloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs, # GAN
                                                                    labels=tf.zeros_like(gQs))) # 0-1
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs, # GAN
                                                                    labels=tf.ones_like(gQs))) # 0-1
    #####################################################
    next_actions_logits = actor(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    gQs2 = discriminator(actions=next_actions_logits, hidden_size=hidden_size, states=next_states, 
                         action_size=action_size, reuse=True)
    gQs2 = tf.reshape(gQs2, shape=[-1]) * (1-dones)
    dloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs2, # GAN
                                                                    labels=tf.zeros_like(gQs2))) # 0-1
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs2, # GAN
                                                                     labels=tf.ones_like(gQs2))) # 0-1
    ##################################################### repeatable!
    next_states_logits = generator(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                                   action_size=action_size, state_size=state_size, reuse=True)
    next_actions_logits = actor(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
                                reuse=True)
    gQs3 = discriminator(actions=next_actions_logits, hidden_size=hidden_size, states=next_states_logits, 
                         action_size=action_size, reuse=True)
    #dones2 = tf.concat(axis=0, values=[dones[1:], tf.ones(shape=[1])])
    gQs3 = tf.reshape(gQs3, shape=[-1]) * (1-dones)
    dloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs3, # GAN
                                                                    labels=tf.zeros_like(gQs3))) # 0-1
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs3, # GAN
                                                                     labels=tf.ones_like(gQs3))) # 0-1
    return actions_logits, aloss, dloss, aloss2

In [8]:
def model_opt(a_loss, d_loss, a_loss2, a_learning_rate, d_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('actor')]
    #g_vars = [var for var in t_vars if var.name.startswith('generator')]
    d_vars = [var for var in t_vars if var.name.startswith('discriminator')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        d_opt = tf.train.AdamOptimizer(d_learning_rate).minimize(d_loss, var_list=d_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, d_opt, a_opt2

In [9]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, d_learning_rate):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.dones, self.rates = model_input(state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.d_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.d_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        d_loss=self.d_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        d_learning_rate=d_learning_rate)

In [10]:
# def sample(buffer, batch_size):
#     idx = np.random.choice(np.arange(len(buffer)), size=batch_size, replace=False)
#     return [buffer[ii] for ii in idx]

In [11]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [12]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [13]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
d_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size: 200/500 a successfull episode size
# gamma = 0.99                   # future reward discount

In [14]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size,
              a_learning_rate=a_learning_rate, 
              d_learning_rate=d_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [15]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        state = env.reset()
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, dloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes average/running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        total_reward = 0 # each episode
        aloss_batch, dloss_batch, aloss2_batch = [], [], []
        state = env.reset() # each episode
        num_step = 0 # each episode
        rate = -1

        # Training steps/batches
        while True:
            # Explore (Env) or Exploit (Model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Rating the memory
            if done is True:
                rate = total_reward/500 # update rate at the end/ when episode is done
                for idx in range(num_step): # episode length
                    if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                        memory.buffer[-1-idx][-1] = rate # rate the trajectory/data
                        
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                #rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                #rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                #if np.count_nonzero(dones)==1 and len(dones) >= 1 and np.max(rates) > 0:
                if len(dones) > 1:
                    # print('np.count_nonzero(dones)==1 and len(dones) >= 1 and np.max(rates) > 0: ', 
                    #       np.count_nonzero(dones), len(dones), np.max(rates))
                    break
            #             if np.count_nonzero(dones)!=1 and len(dones) < 1 and np.max(rates) <= 0:
            #                 print(np.count_nonzero(dones), len(dones), np.max(rates))
            #                 break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                feed_dict = {model.states: states, 
                                            model.actions: actions,
                                            model.next_states: next_states,
                                            #model.rewards: rewards,
                                            model.dones: dones,
                                            model.rates: rates})
            dloss, _ = sess.run([model.d_loss, model.d_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               #model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              #model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print('dones:', 
            #       len(dones), np.count_nonzero(dones), 
            #       len(dones1), np.count_nonzero(dones1), 
            #       len(dones2), np.count_nonzero(dones2))
            aloss_batch.append(aloss)
            dloss_batch.append(dloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'dloss:{:.4f}'.format(np.mean(dloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        dloss_list.append([ep, np.mean(dloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:13.0000 R:13.0000 rate:0.0260 aloss:1.4093 dloss:2.5746 aloss2:2.0963 exploreP:0.9987
Episode:1 meanR:25.5000 R:38.0000 rate:0.0760 aloss:1.4076 dloss:2.6228 aloss2:2.0727 exploreP:0.9950
Episode:2 meanR:21.6667 R:14.0000 rate:0.0280 aloss:1.3970 dloss:2.6274 aloss2:2.0696 exploreP:0.9936
Episode:3 meanR:20.5000 R:17.0000 rate:0.0340 aloss:1.4124 dloss:2.5719 aloss2:2.1059 exploreP:0.9919
Episode:4 meanR:19.0000 R:13.0000 rate:0.0260 aloss:1.4153 dloss:2.5855 aloss2:2.1006 exploreP:0.9906
Episode:5 meanR:26.5000 R:64.0000 rate:0.1280 aloss:1.4082 dloss:2.5080 aloss2:2.1673 exploreP:0.9844
Episode:6 meanR:25.8571 R:22.0000 rate:0.0440 aloss:1.4076 dloss:2.4622 aloss2:2.2116 exploreP:0.9822
Episode:7 meanR:34.7500 R:97.0000 rate:0.1940 aloss:1.4090 dloss:2.4025 aloss2:2.2813 exploreP:0.9729
Episode:8 meanR:32.8889 R:18.0000 rate:0.0360 aloss:1.4125 dloss:2.3422 aloss2:2.3296 exploreP:0.9711
Episode:9 meanR:31.7000 R:21.0000 rate:0.0420 aloss:1.4183 dloss:2.3212 aloss2:2.3

Episode:80 meanR:22.6420 R:39.0000 rate:0.0780 aloss:1.4597 dloss:0.8142 aloss2:7.5504 exploreP:0.8341
Episode:81 meanR:22.5488 R:15.0000 rate:0.0300 aloss:1.4529 dloss:0.7883 aloss2:7.6253 exploreP:0.8329
Episode:82 meanR:22.4217 R:12.0000 rate:0.0240 aloss:1.4290 dloss:0.7705 aloss2:7.6828 exploreP:0.8319
Episode:83 meanR:22.3452 R:16.0000 rate:0.0320 aloss:1.4419 dloss:0.7861 aloss2:7.6626 exploreP:0.8306
Episode:84 meanR:22.2706 R:16.0000 rate:0.0320 aloss:1.4743 dloss:0.7909 aloss2:7.7935 exploreP:0.8293
Episode:85 meanR:22.5233 R:44.0000 rate:0.0880 aloss:1.4570 dloss:0.7702 aloss2:7.7266 exploreP:0.8257
Episode:86 meanR:22.7586 R:43.0000 rate:0.0860 aloss:1.4527 dloss:0.7633 aloss2:7.8605 exploreP:0.8222
Episode:87 meanR:22.7500 R:22.0000 rate:0.0440 aloss:1.4531 dloss:0.7423 aloss2:8.0928 exploreP:0.8204
Episode:88 meanR:22.7079 R:19.0000 rate:0.0380 aloss:1.4473 dloss:0.7485 aloss2:8.2003 exploreP:0.8188
Episode:89 meanR:22.5778 R:11.0000 rate:0.0220 aloss:1.4568 dloss:0.7363 

Episode:159 meanR:20.6000 R:15.0000 rate:0.0300 aloss:1.4638 dloss:0.6074 aloss2:10.6749 exploreP:0.7115
Episode:160 meanR:20.8500 R:34.0000 rate:0.0680 aloss:1.4080 dloss:0.5852 aloss2:10.5429 exploreP:0.7091
Episode:161 meanR:20.6300 R:17.0000 rate:0.0340 aloss:1.4228 dloss:0.5799 aloss2:10.4683 exploreP:0.7079
Episode:162 meanR:20.6100 R:13.0000 rate:0.0260 aloss:1.4386 dloss:0.6357 aloss2:10.0064 exploreP:0.7070
Episode:163 meanR:20.7100 R:24.0000 rate:0.0480 aloss:1.4214 dloss:0.6356 aloss2:10.3623 exploreP:0.7053
Episode:164 meanR:20.6900 R:18.0000 rate:0.0360 aloss:1.4161 dloss:0.6092 aloss2:10.4915 exploreP:0.7041
Episode:165 meanR:20.7500 R:16.0000 rate:0.0320 aloss:1.4001 dloss:0.5799 aloss2:10.2978 exploreP:0.7030
Episode:166 meanR:20.5300 R:9.0000 rate:0.0180 aloss:1.4207 dloss:0.6118 aloss2:10.2101 exploreP:0.7024
Episode:167 meanR:20.3200 R:16.0000 rate:0.0320 aloss:1.4127 dloss:0.5863 aloss2:10.5511 exploreP:0.7013
Episode:168 meanR:20.2400 R:17.0000 rate:0.0340 aloss:1.

Episode:238 meanR:18.2500 R:12.0000 rate:0.0240 aloss:1.4136 dloss:0.5240 aloss2:10.4755 exploreP:0.6216
Episode:239 meanR:17.9700 R:15.0000 rate:0.0300 aloss:1.3836 dloss:0.5685 aloss2:10.5366 exploreP:0.6206
Episode:240 meanR:18.0700 R:20.0000 rate:0.0400 aloss:1.4036 dloss:0.5961 aloss2:10.6368 exploreP:0.6194
Episode:241 meanR:17.8300 R:13.0000 rate:0.0260 aloss:1.4183 dloss:0.6115 aloss2:10.3427 exploreP:0.6186
Episode:242 meanR:17.8000 R:11.0000 rate:0.0220 aloss:1.4083 dloss:0.6348 aloss2:10.3821 exploreP:0.6180
Episode:243 meanR:17.7900 R:12.0000 rate:0.0240 aloss:1.4006 dloss:0.5825 aloss2:10.5921 exploreP:0.6172
Episode:244 meanR:17.8700 R:20.0000 rate:0.0400 aloss:1.4092 dloss:0.5939 aloss2:10.5098 exploreP:0.6160
Episode:245 meanR:17.8900 R:25.0000 rate:0.0500 aloss:1.3987 dloss:0.5932 aloss2:10.3714 exploreP:0.6145
Episode:246 meanR:18.0400 R:30.0000 rate:0.0600 aloss:1.3993 dloss:0.5834 aloss2:10.5460 exploreP:0.6127
Episode:247 meanR:17.8100 R:30.0000 rate:0.0600 aloss:1

Episode:317 meanR:33.3400 R:40.0000 rate:0.0800 aloss:1.3945 dloss:0.5898 aloss2:10.4385 exploreP:0.4645
Episode:318 meanR:33.6800 R:45.0000 rate:0.0900 aloss:1.3995 dloss:0.5685 aloss2:10.3384 exploreP:0.4625
Episode:319 meanR:33.8000 R:24.0000 rate:0.0480 aloss:1.4162 dloss:0.5675 aloss2:10.4244 exploreP:0.4614
Episode:320 meanR:34.0900 R:48.0000 rate:0.0960 aloss:1.4181 dloss:0.5651 aloss2:10.4482 exploreP:0.4593
Episode:321 meanR:34.9200 R:92.0000 rate:0.1840 aloss:1.4044 dloss:0.5809 aloss2:10.4008 exploreP:0.4551
Episode:322 meanR:35.1200 R:35.0000 rate:0.0700 aloss:1.4066 dloss:0.5825 aloss2:10.4969 exploreP:0.4536
Episode:323 meanR:35.2900 R:34.0000 rate:0.0680 aloss:1.4676 dloss:0.5745 aloss2:10.4260 exploreP:0.4521
Episode:324 meanR:35.4600 R:30.0000 rate:0.0600 aloss:1.4345 dloss:0.5935 aloss2:10.5097 exploreP:0.4508
Episode:325 meanR:36.1400 R:99.0000 rate:0.1980 aloss:1.3987 dloss:0.5990 aloss2:10.3815 exploreP:0.4464
Episode:326 meanR:36.5000 R:50.0000 rate:0.1000 aloss:1

...

Episode:5761 meanR:66.8900 R:86.0000 rate:0.1720 aloss:0.9693 dloss:0.6234 aloss2:35.1162 exploreP:0.0100
Episode:5762 meanR:66.8200 R:38.0000 rate:0.0760 aloss:0.9704 dloss:0.6180 aloss2:36.0016 exploreP:0.0100
Episode:5763 meanR:67.4600 R:107.0000 rate:0.2140 aloss:0.9640 dloss:0.6319 aloss2:34.9217 exploreP:0.0100
Episode:5764 meanR:67.3800 R:45.0000 rate:0.0900 aloss:0.9625 dloss:0.6336 aloss2:34.7386 exploreP:0.0100
Episode:5765 meanR:67.2900 R:70.0000 rate:0.1400 aloss:0.9666 dloss:0.6111 aloss2:35.2278 exploreP:0.0100
Episode:5766 meanR:67.1400 R:56.0000 rate:0.1120 aloss:0.9594 dloss:0.6185 aloss2:34.5122 exploreP:0.0100
Episode:5767 meanR:66.8900 R:56.0000 rate:0.1120 aloss:0.9596 dloss:0.6313 aloss2:35.6400 exploreP:0.0100
Episode:5768 meanR:66.6300 R:45.0000 rate:0.0900 aloss:0.9593 dloss:0.6168 aloss2:33.6931 exploreP:0.0100
Episode:5769 meanR:67.8100 R:168.0000 rate:0.3360 aloss:0.9611 dloss:0.6177 aloss2:35.2875 exploreP:0.0100
Episode:5770 meanR:68.5400 R:116.0000 rate:0

Episode:5839 meanR:63.5200 R:39.0000 rate:0.0780 aloss:0.9616 dloss:0.6106 aloss2:34.9080 exploreP:0.0100
Episode:5840 meanR:63.9300 R:77.0000 rate:0.1540 aloss:0.9654 dloss:0.6097 aloss2:36.5762 exploreP:0.0100
Episode:5841 meanR:63.8600 R:45.0000 rate:0.0900 aloss:0.9618 dloss:0.6021 aloss2:36.4046 exploreP:0.0100
Episode:5842 meanR:63.0100 R:38.0000 rate:0.0760 aloss:0.9650 dloss:0.6281 aloss2:34.8845 exploreP:0.0100
Episode:5843 meanR:62.9600 R:55.0000 rate:0.1100 aloss:0.9675 dloss:0.6178 aloss2:36.3196 exploreP:0.0100
Episode:5844 meanR:63.0200 R:95.0000 rate:0.1900 aloss:0.9692 dloss:0.6263 aloss2:37.7048 exploreP:0.0100
Episode:5845 meanR:62.9100 R:36.0000 rate:0.0720 aloss:0.9633 dloss:0.6484 aloss2:35.0865 exploreP:0.0100
Episode:5846 meanR:62.3200 R:40.0000 rate:0.0800 aloss:0.9597 dloss:0.6404 aloss2:36.2352 exploreP:0.0100
Episode:5847 meanR:62.2500 R:48.0000 rate:0.0960 aloss:0.9630 dloss:0.6297 aloss2:38.4031 exploreP:0.0100
Episode:5848 meanR:62.2800 R:70.0000 rate:0.14

Episode:5917 meanR:62.3500 R:41.0000 rate:0.0820 aloss:0.9457 dloss:0.6133 aloss2:40.9731 exploreP:0.0100
Episode:5918 meanR:62.7700 R:124.0000 rate:0.2480 aloss:0.9362 dloss:0.6048 aloss2:43.5135 exploreP:0.0100
Episode:5919 meanR:62.6100 R:42.0000 rate:0.0840 aloss:0.9245 dloss:0.5882 aloss2:40.9333 exploreP:0.0100
Episode:5920 meanR:62.9600 R:80.0000 rate:0.1600 aloss:0.9276 dloss:0.6044 aloss2:42.1108 exploreP:0.0100
Episode:5921 meanR:62.9300 R:39.0000 rate:0.0780 aloss:0.9148 dloss:0.5902 aloss2:41.6463 exploreP:0.0100
Episode:5922 meanR:62.9000 R:48.0000 rate:0.0960 aloss:0.9302 dloss:0.6345 aloss2:44.1067 exploreP:0.0100
Episode:5923 meanR:64.5400 R:204.0000 rate:0.4080 aloss:0.9336 dloss:0.5982 aloss2:42.1803 exploreP:0.0100
Episode:5924 meanR:65.2400 R:113.0000 rate:0.2260 aloss:0.9555 dloss:0.6048 aloss2:42.6946 exploreP:0.0100
Episode:5925 meanR:65.0800 R:38.0000 rate:0.0760 aloss:0.9338 dloss:0.5926 aloss2:41.6844 exploreP:0.0100
Episode:5926 meanR:65.4400 R:92.0000 rate:0

Episode:5995 meanR:69.7400 R:69.0000 rate:0.1380 aloss:0.9563 dloss:0.5856 aloss2:42.9897 exploreP:0.0100
Episode:5996 meanR:70.1900 R:94.0000 rate:0.1880 aloss:0.9686 dloss:0.5942 aloss2:42.0276 exploreP:0.0100
Episode:5997 meanR:70.3500 R:74.0000 rate:0.1480 aloss:0.9730 dloss:0.5945 aloss2:41.5900 exploreP:0.0100
Episode:5998 meanR:70.5200 R:62.0000 rate:0.1240 aloss:0.9678 dloss:0.6038 aloss2:40.6142 exploreP:0.0100
Episode:5999 meanR:70.2300 R:74.0000 rate:0.1480 aloss:1.0117 dloss:0.5954 aloss2:42.3773 exploreP:0.0100
Episode:6000 meanR:70.5100 R:66.0000 rate:0.1320 aloss:1.0058 dloss:0.5946 aloss2:41.5064 exploreP:0.0100
Episode:6001 meanR:70.9000 R:84.0000 rate:0.1680 aloss:0.9644 dloss:0.5847 aloss2:40.3999 exploreP:0.0100
Episode:6002 meanR:70.7100 R:57.0000 rate:0.1140 aloss:1.0084 dloss:0.5954 aloss2:40.4322 exploreP:0.0100
Episode:6003 meanR:71.1800 R:101.0000 rate:0.2020 aloss:0.9882 dloss:0.5889 aloss2:39.7160 exploreP:0.0100
Episode:6004 meanR:71.7000 R:105.0000 rate:0.

Episode:6073 meanR:72.2100 R:91.0000 rate:0.1820 aloss:1.0725 dloss:0.6284 aloss2:33.5411 exploreP:0.0100
Episode:6074 meanR:72.6100 R:95.0000 rate:0.1900 aloss:1.0741 dloss:0.6209 aloss2:34.9205 exploreP:0.0100
Episode:6075 meanR:73.4700 R:128.0000 rate:0.2560 aloss:1.0432 dloss:0.6110 aloss2:34.2751 exploreP:0.0100
Episode:6076 meanR:73.3600 R:74.0000 rate:0.1480 aloss:1.0474 dloss:0.6113 aloss2:33.2665 exploreP:0.0100
Episode:6077 meanR:73.5100 R:80.0000 rate:0.1600 aloss:1.0566 dloss:0.6168 aloss2:32.5489 exploreP:0.0100
Episode:6078 meanR:73.6700 R:102.0000 rate:0.2040 aloss:1.0537 dloss:0.6186 aloss2:34.1007 exploreP:0.0100
Episode:6079 meanR:74.1400 R:111.0000 rate:0.2220 aloss:1.0703 dloss:0.6220 aloss2:32.3993 exploreP:0.0100
Episode:6080 meanR:74.4400 R:101.0000 rate:0.2020 aloss:1.0474 dloss:0.6260 aloss2:32.8264 exploreP:0.0100
Episode:6081 meanR:75.2200 R:143.0000 rate:0.2860 aloss:1.0564 dloss:0.6288 aloss2:32.7128 exploreP:0.0100
Episode:6082 meanR:75.5800 R:93.0000 rate

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('A losses')

In [ ]:
eps, arr = np.array(dloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('D losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('A losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [39]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
